In [12]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd


In [13]:
chromedriver = '/Users/sku11/Downloads/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)


1. instargram에 접속하여  로그인    
|

In [167]:
insta_url = 'https://www.instagram.com/'
driver.get(insta_url)
time.sleep(1)

In [168]:
with open('password.txt') as f:
    password = f.read(11)

In [14]:
driver.close

<bound method WebDriver.close of <selenium.webdriver.chrome.webdriver.WebDriver (session="5ac0d085bce1a675fc6b38654a663a77")>>

In [169]:
email = 'sku1118@nate.com'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)



In [170]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [171]:

try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [172]:
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

###제주도 맛집으로 검색

In [174]:
from urllib.parse import quote
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'

In [175]:
driver.get(url)
time.sleep(5)

### 3. 첫번째 게시글 열기

In [176]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(5)

### 4. 게시글 정보 가져오기

In [177]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [178]:
import unicodedata
try:    #게시글이 없거나 인코딩이 필요한 경우 아래 코드로 실행
    content = soup.select_one('div.C4VMK > span').text
    contant = unicodedata.normalize('NFC', content) 
except:
    content = ' '
content

'제주의 낮과밤은 언제나 즐거움이 가득해요 선선한 날씨라 야외테이블을 이용하기 딱 좋은 시즌이에요 오늘 같은 날 평생테이블에 앉아 맛있는 갈치조림, 활어모듬회 맛보세요!! 제주여행중거나 계획하고 계신다면 이곳을 빼 놓으면 안되겠죠?? 오픈 10시30분 라스트오더 21시 마감 22시 위치 #애월북서길54 #애월하미 전화문의 070 4566 9000 #애월맛집 #제주맛집 #제주도맛집#애월통갈치조림#애월통갈치조림맛집#애월횟집 #제주횟집#제주야외맛집#제주야외식당#제주도갈치조림전문점#제주도통갈치조림맛집#제주도갈치구이맛집#애월하미 #제주여행#제주도존맛'

### 본문 내용에서 해쉬태그 가져오기 , 정규식은 #[^]

In [179]:
import re   
tags =re.findall(r'#[^\s#,\\]+', content)
 

In [180]:
#작성 일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-18'

In [181]:
# 좋아요 수 가져오기  : 좋아요 1,296개
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0


In [182]:
# 5) 위치정보 가져오기
try:
    place = soup.select('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ' '

In [183]:
row = [content, date, like, place, tags]
row

['제주의 낮과밤은 언제나 즐거움이 가득해요 선선한 날씨라 야외테이블을 이용하기 딱 좋은 시즌이에요 오늘 같은 날 평생테이블에 앉아 맛있는 갈치조림, 활어모듬회 맛보세요!! 제주여행중거나 계획하고 계신다면 이곳을 빼 놓으면 안되겠죠?? 오픈 10시30분 라스트오더 21시 마감 22시 위치 #애월북서길54 #애월하미 전화문의 070 4566 9000 #애월맛집 #제주맛집 #제주도맛집#애월통갈치조림#애월통갈치조림맛집#애월횟집 #제주횟집#제주야외맛집#제주야외식당#제주도갈치조림전문점#제주도통갈치조림맛집#제주도갈치구이맛집#애월하미 #제주여행#제주도존맛',
 '2021-08-18',
 0,
 '애월하미',
 ['#애월북서길54',
  '#애월하미',
  '#애월맛집',
  '#제주맛집',
  '#제주도맛집',
  '#애월통갈치조림',
  '#애월통갈치조림맛집',
  '#애월횟집',
  '#제주횟집',
  '#제주야외맛집',
  '#제주야외식당',
  '#제주도갈치조림전문점',
  '#제주도통갈치조림맛집',
  '#제주도갈치구이맛집',
  '#애월하미',
  '#제주여행',
  '#제주도존맛']]

###  5. 다음 게시글로 이동

In [184]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [185]:
#6을 하기 위해 팝업 화면을 없앰(즉, X를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
time.sleep(2)

### 6. 여러개의 게시글 가져오기

In [186]:
from tqdm.notebook import tqdm

In [192]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(5)


# 함수로 만들어서 사용하기

In [193]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0
    try:
        place = soup.select('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''

    return [content, tags, date, like, place]
    

In [ ]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [ ]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

In [ ]:
df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
df.head()

책에 있는 내용으로

In [ ]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(8)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(5)
    
    results = []
    for _ in tqdm(range(200)):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
    df = pd.DataFrame(results, columns=['content','date','like','place','tags'])
    df.to_csv(f'{keyword}.csv', index=False)

    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
    time.sleep(2)
 

4가지 CSV 파일의 중복을 제거한 후 통합저장

In [34]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    df = pd.read_csv(f'jeju_data/{keyword}.csv')
    print(df.shape)

(200, 5)
(200, 5)
(196, 5)
(114, 5)
